[View in Colaboratory](https://colab.research.google.com/github/monuminu/affine_hackathon/blob/master/bag_of_words_meet_bag_of_pop_corn.ipynb)

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"monuirctc","key":"4445ac2b83d3b9b6c3ac5b1811d01baf"}'}

In [0]:
import pandas as pd
import numpy as np
import lxml
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!pip install kaggle

    100% |████████████████████████████████| 61kB 5.6MB/s 
    100% |████████████████████████████████| 235kB 15.9MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/44/2c/df/22a6eeb780c36c28190faef6252b739fdc47145fd87a6642d4
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify


In [8]:
!kaggle competitions download -c word2vec-nlp-tutorial

  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 69.1MB/s]
 64% 17.0M/26.4M [00:01<00:00, 10.1MB/s]
100% 26.4M/26.4M [00:01<00:00, 23.2MB/s]
 39% 5.00M/12.8M [00:00<00:00, 12.0MB/s]
100% 12.8M/12.8M [00:00<00:00, 28.8MB/s]
 38% 5.00M/13.1M [00:00<00:00, 11.8MB/s]
100% 13.1M/13.1M [00:00<00:00, 25.7MB/s]


In [19]:
!ls

kaggle.json		  sample_data		testData.tsv.zip
labeledTrainData.tsv	  sampleSubmission.csv	unlabeledTrainData.tsv
labeledTrainData.tsv.zip  testData.tsv		unlabeledTrainData.tsv.zip


In [18]:
!unzip labeledTrainData.tsv.zip
!unzip unlabeledTrainData.tsv.zip
!unzip testData.tsv.zip

Archive:  testData.tsv.zip
  inflating: testData.tsv            


In [22]:
df_train = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
df_test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=3)
df_train.head()
df_test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [0]:
def review_to_words( raw_review ):
    # 1. Remove HTML
    review_text = raw_review#BeautifulSoup(raw_review, 'html5lib').get_text() 
    
    # 2. Remove non-letters with regex
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                           
    
    # 4. Create set of stopwords
    stops = set(stopwords.words("english"))                  
    
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

df_train['review']=df_train['review'].apply(review_to_words)
df_test["review"]=df_test["review"].apply(review_to_words)

In [42]:
df_train.head()

,id,sentiment,review
0,"""5814_8""",1,stuff going moment mj started listening music ...
1,"""2381_9""",1,classic war worlds timothy hines entertaining ...
2,"""7759_3""",0,film starts manager nicholas bell giving welco...
3,"""3630_4""",0,must assumed praised film greatest filmed oper...
4,"""9495_8""",1,superbly trashy wondrously unpretentious explo...


In [43]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [0]:
list_classes = ["sentiment"]
y = df_train[list_classes].values
list_sentences_train = df_train["review"]
list_sentences_test = df_test["review"]

In [0]:
max_features = 6000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [0]:
maxlen = 370
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

inp = Input(shape=(maxlen, ))
embed_size = 128
x = Embedding(max_features, embed_size)(inp)

In [48]:
x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

batch_size = 32
epochs = 2
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/2
20000/20000 [==============================] - 608s 30ms/step - loss: 0.3793 - acc: 0.8285 - val_loss: 0.3121 - val_acc: 0.8724
Epoch 2/2
20000/20000 [==============================] - 608s 30ms/step - loss: 0.2260 - acc: 0.9112 - val_loss: 0.3233 - val_acc: 0.8654


In [50]:
!ls

kaggle.json		  sample_data		testData.tsv.zip
labeledTrainData.tsv	  sampleSubmission.csv	unlabeledTrainData.tsv
labeledTrainData.tsv.zip  testData.tsv		unlabeledTrainData.tsv.zip


In [51]:
df_test.head(5)

,id,review
0,"""12311_10""",naturally film main themes mortality nostalgia...
1,"""8348_2""",movie disaster within disaster film full great...
2,"""5828_4""",movie kids saw tonight child loved one point k...
3,"""7186_2""",afraid dark left impression several different ...
4,"""12128_7""",accurate depiction small time mob life filmed ...


In [57]:
"12311_10".strip('"').split("_")[1]

'10'

In [0]:
df_test["sentiment"] =  df_test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)
y_test = df_test["sentiment"]

In [0]:
prediction = model.predict(X_te)
y_pred = (prediction > 0.5)

In [62]:
from sklearn.metrics import f1_score, confusion_matrix
print('F1-score: {0}'.format(f1_score(y_pred, y_test)))
print('Confusion matrix:')
confusion_matrix(y_pred, y_test)

F1-score: 0.8504242833621819
Confusion matrix:


array([[11385,  2428],
       [ 1115, 10072]])